In [ ]:
cd /kaggle

In [ ]:
cd input/faces-npz-file/

In [ ]:
# cd input/celeba-dataset/img_align_celeba/

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

In [ ]:
len(os.listdir("img_align_celeba/"))

In [ ]:
sample = mpimg.imread("img_align_celeba/"+os.listdir("img_align_celeba/")[0])
plt.imshow(sample)

In [ ]:
plt.figure(figsize = (10, 15))
for i in range(5*5):
    plt.subplot(5,5,i+1)
    plt.axis("off")
    plt.imshow(mpimg.imread("img_align_celeba/" + os.listdir("img_align_celeba/")[i+10]))
    

In [ ]:
# def load_faces():
#     data = []
#     dir = "img_align_celeba/"
#     for i in os.listdir(dir)[:50000]:
#         pixels = mpimg.imread(dir + i)
#         data.append(pixels)
#     return data
    

In [ ]:
# data = load_faces()

In [ ]:
!pip install mtcnn

In [ ]:
import mtcnn

In [ ]:
from numpy import savez_compressed
from numpy import asarray

In [ ]:
print(mtcnn.__version__)

In [ ]:
model = mtcnn.MTCNN()

In [ ]:
face = model.detect_faces(sample)
x1, y1, width, height = face[0]["box"]

In [ ]:
# def extract_face(model, pixels, dsize = (80,80)):
#     face = model.detect_faces(pixels)
#     x1, y1, width, height = face[0]["box"]
#     x1, y1 = abs(x1), abs(y1)
    
#     x2,y2 = x1+width, y1+height
#     face_pixels = face[y1:y2, x1:x2]
#     image = Image.fromarray(face_pixels)
#     image = image.resize(dsize)
#     face_array = asarray(image)
#     return face_array

In [ ]:
def extract_face(model, pixels, required_size=(80, 80)):
	# detect face in the image
	faces = model.detect_faces(pixels)
	# skip cases where we could not detect a face
	if len(faces) == 0:
		return None
	# extract details of the face
	x1, y1, width, height = faces[0]['box']
	# force detected pixel values to be positive (bug fix)
	x1, y1 = abs(x1), abs(y1)
	# convert into coordinates
	x2, y2 = x1 + width, y1 + height
	# retrieve face pixels
	face_pixels = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face_pixels)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

In [ ]:
def load_image(filename):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	return pixels

In [ ]:
# def load_faces():
#     model = mtcnn.MTCNN()
#     faces = []
#     dir = "img_align_celeba/"
#     for i in os.listdir(dir)[:50000]:
#         pixels = mpimg.imread(dir + i)
#         face = extract_face(model, pixels)
#         faces.append(face)`a
#         print(len(faces), face.shape)
#     return faces

In [ ]:
def load_faces(directory, n_faces):
	# prepare model
	model = mtcnn.MTCNN()
	faces = list()
	# enumerate files
	for filename in os.listdir(directory):
		# load the image
		pixels = load_image(directory + filename)
		# get face
		face = extract_face(model, pixels)
		if face is None:
			continue
		# store
		faces.append(face)
		print(len(faces), face.shape)
		# stop once we have enough
		if len(faces) >= n_faces:
			break
	return asarray(faces)

In [ ]:
dataset = load_faces('img_align_celeba/', 50000)

In [ ]:
cd /kaggle/

In [ ]:
cd working/

In [ ]:
ls

In [ ]:
savez_compressed('img_align.npz', dataset)

In [ ]:
from numpy import load
data = load("img_align.npz")

In [ ]:
faces = data['arr_0']

In [ ]:
faces.shape

In [ ]:
def discriminator(input_shape = (80,80,3)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(128,3, strides = (2,2), padding="same", input_shape = input_shape),
        tf.keras.layers.LeakyReLU(0.2),
        
        tf.keras.layers.Conv2D(128, 3 , strides = (2,2), padding = "same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(128, 3 , strides = (2,2), padding = "same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(256, 3 , strides = (2,2), padding = "same"),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1, "sigmoid")
        
    ])
    model.compile(optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.5), loss = "binary_crossentropy", metrics = ['acc'])
    return model

In [ ]:
def generator(latent_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128 * 5 * 5, input_dim = latent_dim ),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Reshape((5, 5, 128)),
        
        tf.keras.layers.Conv2DTranspose(128, (3,3), strides = (2,2), padding = "same"),
        tf.keras.layers.LeakyReLU(0.2),
        
        tf.keras.layers.Conv2DTranspose(128, (3,3), strides = (2,2), padding = "same"),
        tf.keras.layers.LeakyReLU(0.2),
        
        tf.keras.layers.Conv2DTranspose(128, (3,3), strides = (2,2), padding = "same"),
        tf.keras.layers.LeakyReLU(0.2),
        
        tf.keras.layers.Conv2DTranspose(128, (3,3), strides = (2,2), padding = "same"),
        tf.keras.layers.LeakyReLU(0.2),
        
        tf.keras.layers.Conv2D(3, (3,3), activation = "tanh", padding = "same")
    ])
    return model

In [ ]:
d_model = discriminator()

In [ ]:
d_model.summary()

In [ ]:
g_model = generator(100)

In [ ]:
g_model.summary()

In [ ]:
def gan(g_model, d_model):
    d_model.trainable = False
    model = tf.keras.models.Sequential([
        g_model,
        d_model
    ])
    model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(0.0002, beta_1 = 0.5))
    return model

In [ ]:
gan_model = gan(g_model, d_model)

In [ ]:
def generate_real_samples(dataset, n_size):
    ind = np.random.randint(0, dataset.shape[0], n_size)
    data = dataset[ind]
    data = (data - 127.5)/127.5
    y = np.ones((n_size, 1))
    return data, y

In [ ]:
def generate_latent_space(n_size, latent_dim):
    points= np.random.randn(n_size * latent_dim)
    points = points.reshape(n_size, latent_dim)
    return points

In [ ]:
def generate_fake_examples(g_model, n_size = 64, latent_dim = 100):
    latent_points = generate_latent_space(n_size, latent_dim)
    preds = g_model.predict(latent_points)
    y = np.zeros((n_size, 1))
    return preds, y

In [ ]:
def plot_figure(dataset):
    plt.figure(figsize = (15,15))
    for i in range(7*7):
        plt.subplot(7,7,i+1)
        plt.axis("off")
        plt.imshow(dataset[i])
    plt.show()

In [ ]:
def summarize_performance(d_model, g_model, dataset, n_size = 64):
    
    X_real, y_real = generate_real_samples(dataset, n_size)
    _, acc_r = d_model.evaluate(X_real, y_real)
    
    X_fake, y_fake = generate_fake_examples(g_model)
    _, acc_f = d_model.evaluate(X_fake, y_fake)
    
    print("Real Acc: {} >> Fake Acc: {}".format(acc_r, acc_f))
    
    plot_figure(X_fake)

In [ ]:
def train(g_model, d_model, gan_model, dataset, epochs = 50, batch_size = 64, latent_dim = 100):
    half_batch = int(batch_size/2)
    batches_per_epoch = int(len(dataset)/batch_size)
    for i in range(epochs):
        for j in range(batches_per_epoch):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            
            X_fake, y_fake = generate_fake_examples(g_model, batch_size, latent_dim)
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            
            X_gan = generate_latent_space(batch_size, latent_dim)
            y_gan = np.ones((batch_size, 1))
            
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            if(j%50 == 0):
                print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                    (i+1, j+1, batches_per_epoch, d_loss1, d_loss2, g_loss))
        if ((i+1) %10 == 0):
            summarize_performance(d_model, g_model, dataset)

In [ ]:
train(g_model, d_model,gan_model, faces)

In [ ]:
summarize_performance(d_model, g_model, faces)

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
cd working/

In [ ]:
g_model.save("generator.h5")